
# Orthogonality and Fourier Analysis

This notebook explains how **orthogonality** underlies **Fourier analysis** using inner products,
orthogonal projections, and visual examples.

---

## 1. Orthogonality in function spaces

In Fourier analysis, functions are treated as vectors in an inner-product space (typically $L^2$).
A standard inner product on $[-\pi,\pi]$ is

$$
\langle f, g \rangle = \int_{-\pi}^{\pi} f(x)\,g(x)\,dx.
$$

Two functions are **orthogonal** if $\langle f,g \rangle = 0$.

---

## 2. Orthogonal Fourier basis

On $[-\pi,\pi]$, the families $\{\sin(nx)\}_{n\ge1}$ and $\{\cos(nx)\}_{n\ge0}$ satisfy, for $n\neq m$,

$$
\langle \sin(nx), \sin(mx) \rangle = 0, \qquad
\langle \cos(nx), \cos(mx) \rangle = 0,
$$

and

$$
\langle \sin(nx), \cos(mx) \rangle = 0.
$$

This is the functional analogue of an orthogonal basis in $\mathbb{R}^n$.

---

## 3. Fourier coefficients as projections

A Fourier series expands a function $f$ as

$$
f(x) = a_0 + \sum_{n=1}^{\infty} a_n\cos(nx)
      + \sum_{n=1}^{\infty} b_n\sin(nx).
$$

Because the basis is orthogonal, each coefficient is a projection:

$$
a_n = \frac{\langle f, \cos(nx) \rangle}
           {\langle \cos(nx), \cos(nx) \rangle},
\qquad
b_n = \frac{\langle f, \sin(nx) \rangle}
           {\langle \sin(nx), \sin(nx) \rangle}.
$$

So Fourier analysis is **orthogonal projection onto frequency modes**.

---

## 4. Energy decomposition (Parseval)

Orthogonality yields the identity

$$
\|f\|_2^2
= \frac{1}{\pi}\int_{-\pi}^{\pi} |f(x)|^2\,dx
= a_0^2 + \sum_{n=1}^{\infty}(a_n^2 + b_n^2).
$$

Energy splits cleanly across frequencies.

---

## 5. Visual illustrations


In [ ]:

from IPython.display import Image, display
from pathlib import Path
ASSET_DIR = Path("img")



### 5.1 Orthogonality via inner products

Off-diagonal values are near zero, confirming orthogonality.


In [ ]:
display(Image(filename=str(ASSET_DIR/"sine_orthogonality_heatmap.png")))


### 5.2 Fourier basis functions

Different frequencies are orthogonal under the $L^2$ inner product.


In [ ]:
display(Image(filename=str(ASSET_DIR/"basis_functions.png")))


### 5.3 Fourier series as orthogonal projection

For the square wave $f(x)=\mathrm{sign}(\sin x)$, the Fourier series is

$$
S_K(x) = \sum_{k=1}^{K}
\frac{4}{\pi}\frac{1}{2k-1}\sin((2k-1)x).
$$

Each partial sum is the best $L^2$ approximation using those basis functions.


In [ ]:
display(Image(filename=str(ASSET_DIR/"square_wave_fourier_approx.png")))


## Key takeaways

1. Fourier analysis relies on an **orthogonal basis** of sine and cosine functions.
2. Fourier coefficients are **inner products**, i.e. projections.
3. Orthogonality explains uniqueness, energy conservation, and optimal approximation.



---

# Interactive verification (numerical inner products)

This section lets you **numerically verify orthogonality** by approximating inner products with a trapezoid rule.

We use the inner product on $[-\pi,\pi]$:
$$
\langle f, g \rangle = \int_{-\pi}^{\pi} f(x)g(x)\,dx.
$$

Notes:
- With discrete sampling, you will see **small numerical errors** (near-zero values instead of exact $0$).
- Increasing the number of sample points generally reduces error.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def inner_product(f, g, x):
    """Approximate <f,g> = integral f(x) g(x) dx using the trapezoid rule."""
    return np.trapezoid(f * g, x)

# Parameters you can tweak
N = 8              # number of basis functions (1..N)
num_points = 20001 # grid resolution; increase to reduce numerical error
a, b = -np.pi, np.pi

x = np.linspace(a, b, num_points)

# Build inner product matrices for sine and cosine bases
S = np.zeros((N, N))   # <sin(nx), sin(mx)>
C = np.zeros((N, N))   # <cos(nx), cos(mx)>
SC = np.zeros((N, N))  # <sin(nx), cos(mx)>

for n in range(1, N + 1):
    sn = np.sin(n * x)
    cn = np.cos(n * x)
    for m in range(1, N + 1):
        sm = np.sin(m * x)
        cm = np.cos(m * x)
        S[n - 1, m - 1] = inner_product(sn, sm, x)
        C[n - 1, m - 1] = inner_product(cn, cm, x)
        SC[n - 1, m - 1] = inner_product(sn, cm, x)

# Display raw values (near-diagonal structure is the key)
np.set_printoptions(precision=6, suppress=True)
print("Sine inner products <sin(nx), sin(mx)> (n,m=1..N):")
print(S)
print("\nCosine inner products <cos(nx), cos(mx)> (n,m=1..N):")
print(C)
print("\nCross inner products <sin(nx), cos(mx)> (n,m=1..N):")
print(SC)



## Heatmap view (easier to see orthogonality)

- Off-diagonals should be near $0$.
- Diagonals are non-zero because $\langle \phi_n, \phi_n \rangle > 0$.


In [ ]:

def show_heatmap(M, title):
    plt.figure(figsize=(6.5, 5.5))
    plt.imshow(M, origin="lower", extent=[1, M.shape[1], 1, M.shape[0]], aspect="auto")
    plt.colorbar()
    plt.xlabel("m")
    plt.ylabel("n")
    plt.title(title)
    plt.tight_layout()
    plt.show()

show_heatmap(S, r"$\langle \sin(nx), \sin(mx)\rangle$")
show_heatmap(C, r"$\langle \cos(nx), \cos(mx)\rangle$")
show_heatmap(SC, r"$\langle \sin(nx), \cos(mx)\rangle$")



## Quantify “how orthogonal” (off-diagonal error)

A simple metric is the maximum absolute off-diagonal entry:
$$
\max_{n\neq m} |\langle \phi_n, \phi_m \rangle|.
$$

This should shrink as you increase `num_points`.


In [ ]:

def max_offdiag_abs(M):
    M2 = M.copy()
    np.fill_diagonal(M2, 0.0)
    return np.max(np.abs(M2))

print("Max |off-diagonal| for S (sin-sin):", max_offdiag_abs(S))
print("Max |off-diagonal| for C (cos-cos):", max_offdiag_abs(C))
print("Max |off-diagonal| for SC (sin-cos):", max_offdiag_abs(SC))



## Interactive projection: compute one Fourier coefficient numerically

For an orthogonal basis function $\phi_n$, the projection coefficient is
$$
c_n = \frac{\langle f, \phi_n \rangle}{\langle \phi_n, \phi_n \rangle}.
$$

Below you can pick a function $f$ and compute a coefficient (and optionally compare multiple).


In [ ]:

# Choose a function f(x)
# Examples:
# f = np.sign(np.sin(x))              # square wave
# f = x                               # saw-like (not periodic smooth)
# f = np.cos(2*x) + 0.5*np.sin(5*x)   # known combination
f = np.cos(2*x) + 0.5*np.sin(5*x)

# Choose a basis type and frequency n
basis = "cos"   # "sin" or "cos"
n = 2

if basis == "sin":
    phi = np.sin(n * x)
elif basis == "cos":
    phi = np.cos(n * x)
else:
    raise ValueError("basis must be 'sin' or 'cos'")

cn = inner_product(f, phi, x) / inner_product(phi, phi, x)
print(f"Estimated coefficient for {basis}({n}x):", cn)

# Visualize f and the fitted single-component approximation cn*phi
plt.figure(figsize=(9, 4))
plt.plot(x, f, label="f(x)")
plt.plot(x, cn * phi, label=f"{cn:.3f} * {basis}({n}x)")
plt.xlim(-np.pi, np.pi)
plt.xlabel("x")
plt.ylabel("value")
plt.title("Single-mode projection (numerical)")
plt.legend()
plt.tight_layout()
plt.show()
